# 🎲 GNU Backgammon Research Environment - Interactive Demo

Welcome to the interactive demonstration of the GNU Backgammon research environment! This notebook allows you to:
- Test different agent types
- Run games interactively
- Analyze results
- Experiment with custom configurations

## 🚀 Getting Started

This notebook works perfectly in:
- **GitHub Codespaces** (click the green "Code" button)
- **Gitpod** (click the Gitpod badge in README)
- **Local Jupyter** (after running `make all`)

All dependencies are automatically installed in cloud environments!

In [ ]:
# Import required libraries
import subprocess
import os
import json
import time
from pathlib import Path

print("🎯 GNU Backgammon Research Environment - Interactive Demo")
print("=" * 60)

# Check if gnubg is installed
try:
    result = subprocess.run(['gnubg', '--version'], capture_output=True, text=True)
    print("✅ GNU Backgammon is installed and ready!")
    print(f"Version info: {result.stdout.strip().split()[0:3]}")
except FileNotFoundError:
    print("❌ GNU Backgammon not found. Please run 'make all' first.")
    print("In cloud environments, this should be automatically installed.")

## 🤖 Available Agent Types

The research environment includes several agent types for experimentation:

- **RandomAgent**: Makes random valid moves (baseline)
- **DebugAgent**: Like RandomAgent but with detailed logging
- **LLMAgent**: Uses external LLM APIs for strategic decisions (requires API keys)
- **LiveCodeAgent**: Asks LLM to generate Python code for moves (experimental)

Let's start with a simple demonstration:

In [ ]:
# Function to run a game and display results
def run_demo_game(agent1="RandomAgent", agent2="DebugAgent", num_games=1, debug=True):
    """Run a demo game with specified agents"""
    print(f"🎮 Running {num_games} game(s): {agent1} vs {agent2}")
    print("-" * 50)
    
    # Prepare command
    cmd = [
        "python3", "main.py",
        "--a1", agent1,
        "--a2", agent2,
        "--n", str(num_games)
    ]
    
    if debug:
        cmd.append("--d")
    
    # Run the game
    start_time = time.time()
    result = subprocess.run(cmd, capture_output=True, text=True)
    end_time = time.time()
    
    print(f"⏱️ Execution time: {end_time - start_time:.2f} seconds")
    print("\n📊 Results:")
    print(result.stdout)
    
    if result.stderr:
        print("⚠️ Warnings/Errors:")
        print(result.stderr)
    
    return result.returncode == 0

# Run a quick demo
success = run_demo_game("RandomAgent", "DebugAgent", 1)
print(f"\n{'✅ Demo completed successfully!' if success else '❌ Demo failed'}")

## 📊 Interactive Agent Comparison

Now let's run multiple games to compare agent performance:

In [ ]:
# Interactive widget for agent selection (works in Jupyter environments)
try:
    from ipywidgets import interact, widgets
    
    @interact
    def compare_agents(
        agent1=widgets.Dropdown(options=['RandomAgent', 'DebugAgent'], value='RandomAgent'),
        agent2=widgets.Dropdown(options=['RandomAgent', 'DebugAgent'], value='DebugAgent'),
        num_games=widgets.IntSlider(min=1, max=10, step=1, value=3)
    ):
        run_demo_game(agent1, agent2, num_games, debug=False)
        
except ImportError:
    print("📝 ipywidgets not available - using static comparison")
    
    # Static comparison
    print("🏆 Agent Tournament - 3 rounds each")
    print("=" * 40)
    
    matchups = [
        ("RandomAgent", "DebugAgent"),
        ("DebugAgent", "RandomAgent"),
        ("RandomAgent", "RandomAgent")
    ]
    
    for i, (a1, a2) in enumerate(matchups, 1):
        print(f"\n🥊 Round {i}:")
        run_demo_game(a1, a2, 3, debug=False)

## 🔍 Game Log Analysis

Let's examine the generated game logs to understand what happened:

In [ ]:
# Analyze generated logs
output_dir = Path("output")

if output_dir.exists():
    log_files = list(output_dir.glob("*.log"))
    
    print(f"📁 Found {len(log_files)} log files:")
    for log_file in log_files[-3:]:  # Show last 3 files
        print(f"  📄 {log_file.name} ({log_file.stat().st_size} bytes)")
    
    if log_files:
        # Show content of the most recent log
        latest_log = max(log_files, key=lambda x: x.stat().st_mtime)
        print(f"\n📋 Content of latest log ({latest_log.name}):")
        print("-" * 50)
        
        with open(latest_log, 'r') as f:
            content = f.read()
            # Show first 1000 characters
            if len(content) > 1000:
                print(content[:1000] + "\n\n... (truncated, see full file for complete log)")
            else:
                print(content)
else:
    print("📁 No output directory found. Run a game first!")

## 🛠️ Custom Agent Development

Here's how you can understand the project structure for developing custom agents:

In [ ]:
# Explore project structure
print("🏗️ Project Structure:")
print("=" * 30)

def show_directory_tree(path, prefix="", max_depth=2, current_depth=0):
    if current_depth >= max_depth:
        return
    
    path = Path(path)
    items = sorted([p for p in path.iterdir() if not p.name.startswith('.')])
    
    for i, item in enumerate(items):
        is_last = i == len(items) - 1
        current_prefix = "└── " if is_last else "├── "
        print(f"{prefix}{current_prefix}{item.name}")
        
        if item.is_dir() and current_depth < max_depth - 1:
            extension = "    " if is_last else "│   "
            show_directory_tree(item, prefix + extension, max_depth, current_depth + 1)

show_directory_tree(".")

print("\n🤖 Agent Classes in src/agents/:")
agents_dir = Path("src/agents")
if agents_dir.exists():
    for agent_file in agents_dir.glob("*_agent.py"):
        print(f"  🔧 {agent_file.name}")
        
print("\n📚 Key Files:")
key_files = {
    "main.py": "Entry point for batch game execution",
    "app.py": "Bridge script for gnubg integration", 
    "src/game.py": "Core game loop implementation",
    "src/agents/base.py": "Abstract base class for agents",
    "Makefile": "Setup and installation commands"
}

for file, description in key_files.items():
    status = "✅" if Path(file).exists() else "❌"
    print(f"  {status} {file}: {description}")

## 🧪 Advanced Features

The environment supports advanced features for research:

In [ ]:
# Show available command options
print("⚙️ Available Command Options:")
print("=" * 35)

help_result = subprocess.run(["python3", "main.py", "--help"], capture_output=True, text=True)
print(help_result.stdout)

print("\n🔬 Research Capabilities:")
capabilities = [
    "🎯 Create custom agents by extending base.Agent class",
    "📊 Run statistical analysis with batch games", 
    "🤖 Integration with LLM APIs for AI-powered agents",
    "📝 Detailed logging for move analysis",
    "🏆 Tournament-style competitions between agents",
    "🔧 Full access to GNU Backgammon engine features"
]

for capability in capabilities:
    print(f"  {capability}")

## 🚀 Next Steps

Now that you've seen the interactive demo, here are some things to try:

### 🎮 More Gaming:
```bash
# Run larger tournaments
python3 main.py --a1 RandomAgent --a2 DebugAgent --n 50

# Try different agent combinations
python3 main.py --a1 DebugAgent --a2 DebugAgent --n 10
```

### 🔧 Development:
1. **Create a custom agent**: Copy `src/agents/random_agent.py` and modify the `choose_move` method
2. **Add your agent**: Update `src/agents/__init__.py` to include your new agent
3. **Test it**: Use your agent name in the `--a1` or `--a2` parameters

### 📊 Research:
- Analyze the log files in the `output/` directory
- Compare win rates between different agents
- Study move patterns and strategies

### 🌐 Cloud Development:
- **GitHub Codespaces**: Full development environment
- **Gitpod**: Alternative cloud IDE
- **GitHub Actions**: Automated testing and demos

Happy researching! 🎲🤖